!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
import matplotlib.pyplot as plt

In [2]:

# ## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
world_size_x = 20
world_size_y = 20

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)

# ## initial brain and position generator

result = calculate_individual_output_weights(individuals)

for indiv in result:
    result[indiv]['position'] = [[x[indiv], y[indiv]]]

In [3]:
## def sum_duplicated_neurons_with_input(res, input_list):
##     '''input_list - list of neuron and its weight'''
# res = copy.copy(individuals)
# dic = {}
# for nr in res:
# #     print(nr)
#     dic[nr] = {}
#     for i in res[nr]:
#         n_output = f'{i.output_type}{i.output_id}'
#         n_input = f'{i.input_type}{i.input_id}'
#         dic[nr][i.differ_neuron] = [n_input, n_output, i.weight]
##     return dic
##     pass

In [4]:
k = 4
n = 0
while k>n: 
    print('n',n)
    for indiv in result:
        print('indiv', indiv)
        x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]
        position_list = []
        for out in result[indiv]['out']:
#             if 'out' in out:
            new_pos = move(out, result[indiv]['out'][out])
            position_list.append(new_pos)
        
        position_list = list(map(sum, zip(*position_list)))
        position_list = make_smaller_(position_list)
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))
                            
        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)
#         print('norm', position_list)
        result[indiv]['position'].append(position_list)
######### if n>=1 proceed with neurons #######        
        if n >= 1:
            result = apply_input(result, indiv)
        else:
            pass
        
    last_pos_list = [result[obj]['position'][-1] for obj in result]
    
    for obj in range(len(last_pos_list)):
        for i_nr, i in enumerate(last_pos_list[obj+1:], start=obj+1):
            if last_pos_list[obj] == i:
                result[i_nr]['position'][-1] = result[i_nr]['position'][-2]     
    n += 1
#     print()        
# result

n 0
indiv 0
indiv 1
indiv 2
indiv 3
n 1
indiv 0
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in0', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13]]
('in1', 0)
[[18, 15], [17, 14], [16, 13

TypeError: 'NoneType' object is not subscriptable